In [1]:
print("hej")

hej


In [2]:
from pytradfri import Gateway
from pytradfri.device import light_control, light
from pytradfri.util import load_json, save_json
from pytradfri.command import Command
from pytradfri.device.light import Light
import uuid
from typing import List
import asyncio
from tqdm import tqdm
import time
import multiprocessing
import atexit
from client import get_api, get_api_async
import  math
import concurrent.futures

In [3]:

GATEWAY_IP = "192.168.1.175"

CONFIG_FILE = "tradfri_standalone_psk.conf"

In [4]:
api = await get_api_async(GATEWAY_IP)
api

<bound method APIFactory.request of <pytradfri.api.aiocoap_api.APIFactory object at 0x7873c21d6a90>>

In [5]:
light_ids = [
  65540, 65541, 65542,
  65543, 65544, 65545,
  65546, 65547, 65553,
  65552, 65551, 65550,
  65549
]

In [6]:
''
gateway = Gateway()
gateway_info = await api(Gateway.get_gateway_info())

# Print the firmware version
print("Gateway Firmware Version:", gateway_info.firmware_version)

Gateway Firmware Version: 1.21.31


In [7]:
devices_command = gateway.get_devices()
devices_commands = await api(devices_command)
devices = await api(devices_commands)
lights = [dev for dev in devices if dev.has_light_control]
lights

[<65536 - TRADFRI bulb 11 (TRADFRI bulb E14 CWS 470lm)>,
 <65540 - TRADFRI bulb 5 (TRADFRI bulb E14 CWS 470lm)>,
 <65537 - TRADFRI bulb (TRADFRI bulb E14 CWS 470lm)>,
 <65550 - TRADFRI bulb 2 (TRADFRI bulb E14 CWS 470lm)>,
 <65542 - TRADFRI bulb 7 (TRADFRI bulb E14 CWS 470lm)>,
 <65552 - Bulb 13 (TRADFRI bulb E14 CWS 470lm)>,
 <65539 - TRADFRI bulb 3 (TRADFRI bulb E14 CWS 470lm)>,
 <65538 - TRADFRI bulb 4 (TRADFRI bulb E14 CWS 470lm)>,
 <65549 - TRADFRI bulb 10 (TRADFRI bulb E14 CWS 470lm)>,
 <65544 - TRADFRI bulb 9 (TRADFRI bulb E14 CWS 470lm)>,
 <65551 - TRADFRI bulb 12 (TRADFRI bulb E14 CWS 470lm)>,
 <65545 - TRADFRI bulb 8 (TRADFRI bulb E14 CWS 470lm)>,
 <65543 - TRADFRI bulb 6 (TRADFRI bulb E14 CWS 470lm)>]

In [8]:
len(lights), len(light_ids), [d for d in devices if not d.has_light_control]

(13, 13, [])

In [9]:
print(light_control.LightControl(lights[5]))

<LightControl for Bulb 13 (1 devices)>


In [10]:
async def exe_time(func):
    start = time.time
    return func(), time.time - start


In [11]:

time.time()

1712927159.292172

In [12]:
12%10

2

In [13]:
async def blink0(light, up_time, down_time, min_dim = 10, max_dim = 100, start_time = None,
                tolerance = 10):
    start_time = start_time if start_time != None else time.time()
   
    a = None
    b = None
    misses = []
    while True:
     

        
        dif1 =  (start_time + down_time) - time.time()
        await asyncio.sleep(max(dif1, 0))
        if a != None:
            await a
        a = api(light_control.LightControl(light).set_dimmer(min_dim))
        
        dif2 = (start_time + down_time + up_time) - time.time()
        await asyncio.sleep(max(dif2, 0))
        if b != None:
            await b
        b = api(light_control.LightControl(light).set_dimmer(max_dim))
        start_time = start_time +  down_time + up_time
        if dif1 == 0 or dif2 == 0:
            misses.append(dif1+dif2)
        else:
            misses = []
        if len(misses) > tolerance:
            print(misses)
            print("To many misses")
            await api(light_control.LightControl(light).set_dimmer(0))
            return

In [14]:

async def blink(light, on_beat, off_beat, min_dim = 10, max_dim = 100):
   
    a = None
    b = None
    while True:
        if a != None:
            await a
        await on_beat.wait()
        a = api(light_control.LightControl(light).set_dimmer(min_dim))
     
        if b != None:
            await b
        await off_beat.wait()
        b = api(light_control.LightControl(light).set_dimmer(max_dim))
        
       
       
    

In [16]:
tasks = []
start_time = time.time() + 1
on_beat = asyncio.Event()
on_beat.clear()
off_beat = asyncio.Event()
off_beat.clear()
async def tick():
    while True:
        await on_beat.set()
        await asyncio.sleep(0.5)
        on_beat.clear()
        off_beat.set()
        await asyncio.sleep(0.5)
        off_beat.clear()
tasks = [tick(), blink(lights[11], on_beat, off_beat, 2, 130)]
await asyncio.gather(*tasks)


In [15]:
tasks = []
for light in lights[::-1]:
    time.sleep(0.0)
    
    tasks.append(api(light_control.LightControl(light).set_dimmer(0)))
    tasks.append(api(light_control.LightControl(light).set_hex_color("ffffff")))

await asyncio.gather(*tasks)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [30]:

b = api(light_control.LightControl(lights[11]).set_dimmer(2))
await b


In [24]:
lights[12]

<65545 - TRADFRI bulb 8 (TRADFRI bulb E14 CWS 470lm)>

65547

In [ ]:
lights[9].id

65550

In [ ]:
[l.id for l in lights]

[65536,
 65537,
 65538,
 65539,
 65540,
 65542,
 65544,
 65545,
 65549,
 65550,
 65543,
 65551,
 65552]